In [2]:
import numpy as np
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

2024-07-08 08:49:42.883485: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-08 08:49:42.912739: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 08:49:43.061300: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 08:49:43.062293: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-08 08:49:43.766243: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
from pandas import read_csv
import pandas as pd

df = read_csv('./data/data.csv')
df['Date & Time'] = pd.to_datetime(df['Date & Time'])
df.set_index('Date & Time', inplace = True)

/tmp/ipykernel_2449/1917579783.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date & Time'] = pd.to_datetime(df['Date & Time'])


In [4]:
cols = ['Inside Temp – C',
 'High Inside Temp – C',
 'Low Inside Temp – C',
 'Inside Hum - %',
 'High Inside Hum - %',
 'Low Inside Hum - %',
 'Inside Dew Point – C',
 'Inside Heat Index – C',
 'Barometer - mb',
 'High Bar - mb',
 'Low Bar - mb',
 'Absolute Pressure - mb',
 'Low Wind Chill – C',
 'Heat Index – C',
 'Heating Degree Days',
 'Cooling Degree Days']

data = df[cols]

data.replace('--', pd.NA, inplace=True)
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43258 entries, 2024-01-23 13:00:00 to 2024-06-28 17:35:00
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Inside Temp – C         43258 non-null  int64  
 1   High Inside Temp – C    43258 non-null  int64  
 2   Low Inside Temp – C     43258 non-null  int64  
 3   Inside Hum - %          43258 non-null  int64  
 4   High Inside Hum - %     43258 non-null  int64  
 5   Low Inside Hum - %      43258 non-null  int64  
 6   Inside Dew Point – C    43258 non-null  int64  
 7   Inside Heat Index – C   43258 non-null  int64  
 8   Barometer - mb          43258 non-null  float64
 9   High Bar - mb           43258 non-null  float64
 10  Low Bar - mb            43258 non-null  float64
 11  Absolute Pressure - mb  43258 non-null  float64
 12  Low Wind Chill – C      43258 non-null  object 
 13  Heat Index – C          43258 non-null  object 
 14  Hea

/tmp/ipykernel_2449/1157609063.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.replace('--', pd.NA, inplace=True)
/tmp/ipykernel_2449/1157609063.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [5]:
dataset = data.values
dataset = dataset.astype('float32')

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0, 1))
dataset = scaler.fit_transform(dataset)

In [7]:
train_size = int(len(dataset) * 0.85)

train, test = dataset[:train_size], dataset[train_size:]

print(train.shape)
print(test.shape)

(36769, 16)
(6489, 16)


In [8]:
def split_sequences(dataset, n_steps = 1):
    x,  y = [], []

    for i in range(len(dataset) - n_steps):
        xs = dataset[i: (i + n_steps)]
        ys = dataset[i + n_steps]

        x.append(xs)
        y.append(ys)

    return np.array(x), np.array(y)

In [9]:
n_steps = 6

train_x, train_y = split_sequences(train, n_steps)
test_x, test_y = split_sequences(test, n_steps)

print(train_x.shape)
print(test_x.shape)

print(train_y.shape)
print(test_y.shape)

print(train_x[0])
print(train_y[0])

test_y = scaler.inverse_transform(test_y)

(36763, 6, 16)
(6483, 6, 16)
(36763, 16)
(6483, 16)
[[0.07692301 0.07692301 0.07692301 0.3076923  0.3023256  0.28205132
  0.16666669 0.         0.75084305 0.7575798  0.7474747  0.746624
  0.25925928 0.18421054 0.         0.6117647 ]
 [0.07692301 0.07692301 0.07692301 0.2051282  0.25581396 0.23076928
  0.05555558 0.         0.7407417  0.7474785  0.73737335 0.73986435
  0.25925928 0.21052629 0.         0.63529414]
 [0.07692301 0.07692301 0.07692301 0.2051282  0.25581396 0.2051282
  0.05555558 0.         0.7306404  0.73737717 0.7306404  0.7297325
  0.2962963  0.21052629 0.         0.6470588 ]
 [0.07692301 0.07692301 0.07692301 0.23076916 0.2093023  0.23076928
  0.1111111  0.04166669 0.7205391  0.7306442  0.7205391  0.71959305
  0.2962963  0.21052629 0.         0.65882355]
 [0.15384614 0.15384614 0.07692301 0.25641024 0.23255813 0.25641024
  0.16666669 0.04166669 0.71717453 0.71717453 0.71717453 0.71621704
  0.2962963  0.21052629 0.         0.6823529 ]
 [0.15384614 0.15384614 0.15384614 0.

In [10]:
n_features = train_x.shape[2]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

history = model.fit(train_x, train_y, epochs=40, verbose=0)

In [16]:
yhat = scaler.inverse_transform(model.predict(test_x, verbose=0)) 
print("Prediction: ")
print(np.round(yhat[:2], decimals = 1))
print("Real: ")
print(np.round(test_y[:2], decimals = 1))

Prediction: 


TypeError: 'float' object cannot be interpreted as an integer

In [14]:
from sklearn.metrics import mean_squared_error

print(mean_squared_error(y_true=test_y, y_pred=yhat))

0.17130911
